# Wikileaks email parsing

This notebook parses the scraped DNC and Sony email data

In [1]:
from collections import defaultdict
from html.parser import HTMLParser
from email.parser import Parser
import os
from pathlib import Path
import base64
#from urllib.parse import unquote
import re

from tqdm import tqdm
from lxml import html
from lxml.etree import tostring
import pandas as pd
from pytz import timezone

email_regex = r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)"

In [2]:
class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.take = True
        self.fed = []
    def handle_data(self, d):
        if self.take:
            self.fed.append(d)
    def handle_starttag(self, tag, attrs):
        if tag == "style":
            self.take = False
    def handle_endtag(self, tag):
        if tag == "style":
            self.take = True
    def get_data(self):
        return ''.join(self.fed)


def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()


def detect_encoding(body):
    if 'charset=' in body:
        pattern = r'charset=(?:3[Dd])?"?([\-\w]+)"?'
        encoding = re.search(pattern, body).group(1)
        return encoding.lower()
    else:
        return 'utf-8'


def base64_decode(body):
    if 'Content-Transfer-Encoding: base64' in body:
        pattern = r'Content-Transfer-Encoding: base64\n\n([\n\w\+\/]+(?:=+|\n))'
        
        body_b64s = re.findall(pattern, body)
        for body_b64 in body_b64s:
            subbody_decoded = base64.b64decode(body_b64).decode('utf-8')
            body = body.replace(body_b64, subbody_decoded)
        return body
    else:
        return body
    
    
def handle_multipart(body):
    if 'Content-Type: multipart/alternative' in body:
        # Extract the boundary substring
        boundary_pattern = r'boundary="([\-\w=.]+)"'
        boundary = re.search(boundary_pattern, body).group(1)
        boundary = r'\n\n--' + boundary

        # Remove the multipart headers from the body
        headers_pattern = r'Content-Type: multipart/alternative;\n\tboundary="[\-\w=.]+"'       
        body_snipped = re.sub(headers_pattern, '', body)
        body_split = [b for b in re.split(boundary, body_snipped) if b]
        true_body = body_split[0]  # The first part is always the email body. The other parts are attachemnts
        return true_body
    else:
        return body
    
    
def remove_headers(body):
    """Handles some quirkiness I saw in the Windows encoding of email bodies"""
    headers = [
        r'Content-ID: <[^>]+>',
        r'Content-Type: \w+\/\w+;?',
        r'Content-Language: [-\w]+',
        r'Content-Transfer-Encoding: [-\w]+',
        r'Content-Disposition: ([\w]+)',
        r'Content-Description: [\w.]+',
        r'Content-Location: [\w.]+',
        r'X-WatchGuard-AntiVirus: part scanned. clean action=allow',
        r'charset="[\-\w]+"\n',
        r'boundary="\w+"'
    ]
    for header in headers:
        body = re.sub(header, '', body)
    body = body.strip()
    return body


# def clean_body(body):
#     """Handles some quirkiness I saw in the Windows encoding of email bodies"""
#     quirks = [
#         (r'Content-Type: \w+\/\w+;', ''),
#         (r'charset="[\-\w]+"\n', ''),
#         (r'Content-Transfer-Encoding: [-\w]+', ''),
#         (r'X-WatchGuard-AntiVirus: part scanned. clean action=allow', ''),
#         (r'Content-Language: [-\w]+', ''),
#         (r'boundary="[-\w\.=]+"', ''),
#         (r'--_\d+_\w+(?=\n)', ''),
#         (r'\w{50,}', '')
#     ]
#     for bad, good in quirks:
#         body = re.sub(bad, good, body)
#     body = body.strip()
#     return body


def parse_email(msg):
    eml_dict = defaultdict(lambda x: None)
    parsed_eml = Parser().parsestr(msg)
    eml_dict.update(parsed_eml)
    
    # Replace the Body with a HTML-stripped string of the body
    body = parsed_eml.get_payload()
    if isinstance(body, list):
        body = body[0].as_string()

    # Handle "multi-part" content/emails
    body = handle_multipart(body)

    encoding = detect_encoding(body)
    def decode(m):
        try:
            # for some reason the NULL byte is forbidden in CSV? the csv standard module raises an exception if it tried to read it...
            if "=00" in m.group():
                return m.group()
            return bytearray.fromhex(m.group().replace("=", "").replace("%", "")).decode(encoding)
        except UnicodeDecodeError:
            return m.group()
        except LookupError:
            return m.group()
    body_decoded = re.sub(r"([=%][A-Fa-f0-9]{2})+", decode, body)
    body_decoded = base64_decode(body_decoded)
    body_decoded = remove_headers(body_decoded)

    # Strip HTML tags
    body_decoded = strip_tags(body_decoded)

    eml_dict['OriginalEncoding'] = encoding
    eml_dict['Body'] = body
    eml_dict['BodyDecoded'] = body_decoded
    
    return eml_dict


def decode_subject(subject):
    encodings = re.search(r'=\?([\w-]+)\?([A-Za-z])\?', subject)
    if encodings is None:
        return subject
    subject_encoding = encodings.group(1)
    subject_xfer_encoding = encodings.group(2)
    subject = re.sub(r'=\?[\w-]+\?[A-Za-z]\?', '', subject).strip()
    if subject_xfer_encoding == "b":
        subject = base64.b64decode(subject).decode()
    if subject_encoding != "us-ascii":
        def decode_bytes(m):
            try:
                return bytearray.fromhex(m.group().replace("=", "")).decode(subject_encoding)
            except:
                print(subject)
                print(m)
                raise e
        subject = re.sub(r"([=%][A-Fa-f0-9]{2})+", decode_bytes, subject)
    return subject

## DNC Parsing

In [7]:
dnc_files = list(Path("dnc_emails_raw").glob("*.eml"))
dnc_files[:5]

[PosixPath('dnc_emails_raw/19296.eml'),
 PosixPath('dnc_emails_raw/21709.eml'),
 PosixPath('dnc_emails_raw/5924.eml'),
 PosixPath('dnc_emails_raw/2093.eml'),
 PosixPath('dnc_emails_raw/18188.eml')]

In [8]:
error_emails = []
errors = []
success_emails = []

for dnc_file in dnc_files:
    try:
        with open(dnc_file, 'r') as f:
            text = f.read()
    except UnicodeDecodeError:
        continue
    try:
        eml_parsed = parse_email(text)
        success_emails.append(eml_parsed)
    except Exception as e:
        errors.append(e)
        error_emails.append(dnc_file)

In [9]:
len(success_emails), len(error_emails)

(22960, 6)

In [10]:
df_raw = pd.DataFrame(success_emails)
cols_of_interest = ['To', 'From', 'Subject', 'BodyDecoded', 'Date']
df = df_raw[cols_of_interest]
df = df.rename(columns={"To": "to",
                        "From": "from",
                        "Subject": "subject",
                        "BodyDecoded": "body",
                        "Date": "date"})
df["body"] = df.body.str.replace(r"=\n", "\n").str.replace(r"(\s*\n){4,}", "\n\n\n\n")
df["subject"] = df.subject.fillna("").apply(decode_subject)
df["id"] = df.index.astype(str)
df["from"] = df["from"].str.extract(email_regex)
df["to"] = df["to"].str.extractall(email_regex).groupby(level=0).agg(list)[0].apply(lambda s: ",".join(s))
df = df.dropna(subset=["to", "from", "date"])
df["date"] = df.date.apply(lambda d: datetime.strptime(d, "%a, %d %b %Y %H:%M:%S %z").astimezone(timezone("UTC")).strftime("%Y-%m-%dT%H:%M:%SZ"))
df

,to,from,subject,body,date,id
0,comers@dnc.org,jetblueairways@email.jetblue.com,We're makin' it rain low fares!,"\n\n\n\nJetBlue\n\n\n\n Hot fares, cool des...",2016-05-03T11:22:21Z,0
1,ReedA@dnc.org,ParrishD@dnc.org,RE: Finance contribution,"Thank you!\n\nFrom: Alan Reed\nSent: Monday, M...",2016-05-09T20:15:22Z,1
2,"PaustenbachM@dnc.org,WalkerE@dnc.org,MirandaL@...",WalshT@dnc.org,RE:_COMMS_REVIEW:_The_“King_of_Debt”_and_Congr...,Edits below – I added a hit on his bankruptcie...,2016-05-09T16:49:39Z,2
3,"JacquelynLopez@perkinscoie.com,FreundlichC@dnc...",LykinsT@dnc.org,RE: For quick approval: FLOTUS,"Good\n\nFrom: Lopez, Jacquelyn K. (Perkins Coi...",2016-05-24T21:16:26Z,3
4,"jklein@hillaryclinton.com,KaplanJ@dnc.org,Parr...",ComerS@dnc.org,RE: Check Inquiry,"Yup, she maxed out on 5/3.\r\n\r\nFrom: Justin...",2016-05-18T14:00:13Z,4
5,"SargeM@dnc.org,Comm_D@dnc.org",PetersonK@dnc.org,RE: Video Request: Trump in 2013 Music Video,"Saved here\n\nFrom: Sarge, Matthew\nSent: Mond...",2016-05-16T19:28:02Z,5
6,"ComerS@dnc.org,MarquezK@dnc.org",ShapiroA@dnc.org,RE: RE: Villanova Wildcats at the White House ...,Thank you!\n\nAlexandra Shapiro\n(858) 361-246...,2016-05-16T21:54:19Z,6
7,VaughnJ@dnc.org,ParrishD@dnc.org,RE: Taylor,"They don't have swiss, but they have provolone...",2016-05-17T14:58:37Z,7
8,Research_D@dnc.org,YoxallC@dnc.org,Event: Nikki Haley Spoke at Clemson University...,Was an honor to speak @ Clemson's Global Tire ...,2016-04-19T15:05:31Z,8
9,Comm_D@dnc.org,SorbieS@dnc.org,Re: Video request: Kasich in McKees Rocks,Also here if that doesn't work:\n\nhttp://m.wt...,2016-04-25T23:17:23Z,9


In [11]:
df.to_csv('dnc_emails.csv', index=False)

## Sony Emails

In [3]:
sony_files = list(Path("sony_emails_raw").glob('*.eml'))
sony_files[:5]

[PosixPath('sony_emails_raw/19296.eml'),
 PosixPath('sony_emails_raw/21709.eml'),
 PosixPath('sony_emails_raw/5924.eml'),
 PosixPath('sony_emails_raw/2093.eml'),
 PosixPath('sony_emails_raw/18188.eml')]

In [4]:
def parse_sony_email(sony_file):
    with open(sony_file, 'r') as f:
        text = f.read()
    tree = html.fromstring(text)
    eml_text = tree.xpath('//div[@id="email_raw"]/pre/text()')
    if eml_text:
        eml_parsed = parse_email(eml_text[0])
        return eml_parsed
    else:
        return False

In [5]:
sony_successes = []
sony_failures = []

for sony_file in sony_files:
    try:
        sony_eml = parse_sony_email(sony_file)
        if sony_eml:
            sony_successes.append(sony_eml)
        else:
            sony_failures.append(sony_file)
    except:
        sony_failures.append(sony_file)

In [6]:
len(sony_successes), len(sony_failures)

(21338, 1661)

In [7]:
sony_df = pd.DataFrame(sony_successes)
cols_of_interest = ['To', 'From', 'Subject', 'BodyDecoded', 'Date']
sony_df = sony_df[cols_of_interest]
sony_df = sony_df.rename(columns={"To": "to",
                        "From": "from",
                        "Subject": "subject",
                        "BodyDecoded": "body",
                        "Date": "date"})
sony_df['body'] = sony_df.body.str.strip()
sony_df['subject'] = sony_df.subject.fillna("").apply(decode_subject)
sony_df["id"] = sony_df.index.astype(str)
sony_df["from"] = sony_df["from"].str.extract(email_regex)
sony_df["to"] = sony_df["to"].str.extractall(email_regex).groupby(level=0).agg(list)[0].apply(lambda s: ",".join(s))
sony_df = sony_df.dropna(subset=["to", "from", "date"])
sony_df["date"] = sony_df.date.apply(lambda d: datetime.strptime(d, "%a, %d %b %Y %H:%M:%S %z").astimezone(timezone("UTC")).strftime("%Y-%m-%dT%H:%M:%SZ"))
sony_df.iloc[:10, :]

,to,from,subject,body,date,id
0,amy_pascal@spe.sony.com,alerts@deadline.com,[Deadline.com] TCA: PBS 'Nova' Panelists Scorc...,[Deadline.com] TCA: PBS 'Nova' Panelists Scorc...,2014-07-24T00:08:43Z,0
3,Amy_Pascal@spe.sony.com,Connor@spe.sony.com,Chappie - Cell Phone/More Info,Chappie - Cell Phone/More Info\n\n\n\n\nThis w...,2014-06-12T21:43:12Z,3
5,Amy_Pascal@spe.sony.com,Adam_North@spe.sony.com,,Just left the office. I'm in San Diego until F...,2013-11-27T21:14:45Z,5
14,amy_pascal@spe.sony.com,order-update@amazon.com,"Your Amazon.com order of ""Daron Skymarks Air F...","Your Amazon.com order of ""Daron Skymarks Air F...",2014-02-23T22:13:50Z,14
16,"Michael_Lynton@spe.sony.com,Amy_Pascal@spe.son...",Jeff_Blake@spe.sony.com,This weekend,This weekend\n\n\n\n\nI'll be at the presentat...,2014-01-31T02:58:51Z,16
17,Amy_Pascal@spe.sony.com,Thomas_Gargotta@spe.sony.com,Checking in...,Checking in...\n\n\n\n\nReel?,2013-11-08T20:16:59Z,17
20,jon@jgprods.com,kanzeon1@kanzeoncorp.com,Re: Fotos by me and coop at paris prem,Re: Fotos by me and coop at paris prem\n\n\n\n...,2014-02-04T03:14:20Z,20
25,"Amy_Pascal@spe.sony.com,Doug_Belgrad@spe.sony....",Lauren_Abrahams@spe.sony.com,UNCHARTED directors list,UNCHARTED directors list\n\n\n\n\nAttached is ...,2013-11-01T23:28:43Z,25
26,amy_pascal@spe.sony.com,ship-confirm@amazon.com,Your Amazon.com order has shipped (#115-652521...,Your Amazon.com order has shipped (#115-652521...,2014-06-13T05:10:08Z,26
31,Steve_Mosko@spe.sony.com,Mark_Rogers@spe.sony.com,Q2 deeper dive,"Steve, I’ve been working on the analysis you a...",2014-11-07T23:06:23Z,31


In [8]:
# sometimes the subject appears in the beginning of the body. if so, remove it
def remove_prefix(row):
    subject = row.subject.replace("\n", "")
    if row.body.startswith(subject):
        row["body"] = row.body[len(subject):].strip()
    return row
sony_df = sony_df.apply(remove_prefix, axis=1)
sony_df.iloc[:10, :]

,to,from,subject,body,date,id
0,amy_pascal@spe.sony.com,alerts@deadline.com,[Deadline.com] TCA: PBS 'Nova' Panelists Scorc...,http://link.deadline.com/4e8c83dc15b30dde9be1f...,2014-07-24T00:08:43Z,0
3,Amy_Pascal@spe.sony.com,Connor@spe.sony.com,Chappie - Cell Phone/More Info,This weekend they are shooting some small pick...,2014-06-12T21:43:12Z,3
5,Amy_Pascal@spe.sony.com,Adam_North@spe.sony.com,,Just left the office. I'm in San Diego until F...,2013-11-27T21:14:45Z,5
14,amy_pascal@spe.sony.com,order-update@amazon.com,"Your Amazon.com order of ""Daron Skymarks Air F...",Amazon.com Your Orders | Your Account | A...,2014-02-23T22:13:50Z,14
16,"Michael_Lynton@spe.sony.com,Amy_Pascal@spe.son...",Jeff_Blake@spe.sony.com,This weekend,I'll be at the presentation Friday . I'm leavi...,2014-01-31T02:58:51Z,16
17,Amy_Pascal@spe.sony.com,Thomas_Gargotta@spe.sony.com,Checking in...,Reel?,2013-11-08T20:16:59Z,17
20,jon@jgprods.com,kanzeon1@kanzeoncorp.com,Re: Fotos by me and coop at paris prem,Social media ? \n\n\n------Original Message---...,2014-02-04T03:14:20Z,20
25,"Amy_Pascal@spe.sony.com,Doug_Belgrad@spe.sony....",Lauren_Abrahams@spe.sony.com,UNCHARTED directors list,"Attached is our list, which we wanted you to h...",2013-11-01T23:28:43Z,25
26,amy_pascal@spe.sony.com,ship-confirm@amazon.com,Your Amazon.com order has shipped (#115-652521...,Amazon Your Orders | Your Account ...,2014-06-13T05:10:08Z,26
31,Steve_Mosko@spe.sony.com,Mark_Rogers@spe.sony.com,Q2 deeper dive,"Steve, I’ve been working on the analysis you a...",2014-11-07T23:06:23Z,31


In [9]:
sony_df.to_csv('sony_emails.csv', index=False)

## Clinton Parsing

In [142]:
cols_of_interest = ['Id', 'MetadataTo', 'MetadataFrom', 'MetadataDateSent',                        
                    'ExtractedSubject', 'ExtractedBodyText']
rename_dict = {
    'Id': 'id',
    'MetadataTo': 'to',
    'MetadataFrom': 'from',
    'MetadataDateSent': 'date',
    'ExtractedSubject': 'subject',
    'ExtractedBodyText': 'body'
}
clinton_df = (pd.read_csv('clinton_raw.csv')
                .loc[:, cols_of_interest]
                .rename(columns=rename_dict)
                .dropna())


In [144]:
clinton_df.to_csv("clinton_emails.csv", index=False)